# Task: Using keras

In [ ]:
import numpy as np
import pandas as pd

### Data

From https://archive.ics.uci.edu/ml/datasets/heart+Disease

> **target**: diagnosis of heart disease (angiographic disease status)
-- Value 0: < 50% diameter narrowing
-- Value 1: > 50% diameter narrowing

> **sex**: sex (1 = male; 0 = female)

> **cp**: chest pain type
-- Value 1: typical angina
-- Value 2: atypical angina
-- Value 3: non-anginal pain
-- Value 4: asymptomatic

In [ ]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"

## read the csv file under the file_url into a pandas dataframe
df = ...

In [ ]:
display(df)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    object 
 13  target    303 non-null    int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 33.3+ KB


In [ ]:
## create a new dataframe where the ["sex", "cp", "thal"] columns are one-hot encoded.

df_new = ...


### Train-valid-test split

In [ ]:
## import scikit-learn's StandardScaler and train_test_split
...

In [ ]:
## create the input array X and target array y from df_new.
## y should be the 1-dimensional numpy array containing the values of the "target" column,
## X should be the 2-dimensional numpy array containing the values of the df_new EXCEPT for hte "target" column.

targetcol = "target"

X = ...
y = ...

## print out the shapes of X and y to check they're compatible.
....

In [ ]:
## Do a shuffled train-test-validation split. 
## Train data (X_train, y_train) should be 70% of the data, 
## test (X_test, y_test) and validation (X_val, y_val) 15-15% each.
## Use random seeding with seed 42 to obtain reproducible results!

...

print("", X_train.shape, "\n", y_train.shape, 
      "\n", X_val.shape, "\n", y_val.shape,
      "\n", X_test.shape, "\n", y_test.shape)

Optional: create scaled data

In [ ]:
## create a standardscaler, fit it on X_train and create a transformed X_..._scaled scaled array from all X splits.
...

In [ ]:
print("", X_train_scaled.shape, "\n", y_train.shape, 
      "\n", X_val_scaled.shape, "\n", y_val.shape,
      "\n", X_test_scaled.shape, "\n", y_test.shape)

In [ ]:
## a little extra: visualization after decomposition
import seaborn as sns
import matplotlib.pyplot as plt

## dimred with PCA into 2D
from sklearn.decomposition import PCA
pca = PCA(2)
X_transformed = pca.fit_transform(X_scaled)
sns.scatterplot(data=pd.DataFrame(X_transformed, columns=["PC1", "PC2"]),
                x="PC1", y="PC2", hue=y);
plt.title("PCA dim. red.")
plt.show()

## dimred with PCA into 1D
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1)
X_transformed = lda.fit_transform(X=X_scaled, y=y)
sns.scatterplot(x=X_transformed[:,0], y=y, hue=y)
plt.xlabel("new_dim")
plt.ylabel("target")
plt.title("LDA dim. red.")
plt.show()

### Keras model using sequential API

In [ ]:
## import the Dense layer and Sequential class from tf.keras.
## import tensorflow with its usual abbreviation.
...
...
...

In [ ]:
## Write the function which creates a keras model using the sequential API.
## The model should have 1 layer, which should have 3 neurons in it, and use the 'sigmoid' activation function.
## (For this first layer, feel free to play around with other activation functions, or even lack of it, 
## which is None, that is simple linear activation.)
## The final (output) layer should have 1 unit, and the 'sigmoid' activation function.
## Then compile the model, using "binary_crossentropy" loss, "Adam" optimizer, and ["accuracy"] metrics

def create_model():

    tf.keras.backend.clear_session()

    model = Sequential()

    ## add the first layer 
    ...

    ## add the output layer
    ...

    ## compile the model
    ...

    return model

In [ ]:
## now write a function which creates a model, prints out a summary, fits it, and plots the history.

def create_and_fit_model(X_train, y_train, X_val, y_val, num_epochs, num_batchsize):
    tf.keras.utils.set_random_seed(42)
    model = create_model()
    
    ## print out model summary
    print(...)

    ## Fit the model and store training history in the history variable.
    ## Fit it on the X_train and y_train data.
    ## Use num_epochs number of epochs, num_batchsize as batch size.
    history = model.fit(...,
                        validation_data=(X_val, y_val),
                        )
                        
    ## Plotting the history is done for you:
    historydf = pd.DataFrame(history.history)
    historydf.plot(xlabel="epoch", secondary_y=[c for c in historydf.columns if c.endswith("loss")]);

    return model, historydf

Model fit parameters

In [ ]:
num_epochs = 100
num_batchsize = 32

#### Do fitting

In [ ]:
model, historydf = create_and_fit_model(X_train, y_train, X_val, y_val, num_epochs, num_batchsize)

Optional: Create a model and fit on scaled data

In [ ]:
model_scaled, historydf_scaled = create_and_fit_model(X_train_scaled, y_train, X_val_scaled, y_val, num_epochs, num_batchsize)

In [ ]:
## No task, just compare the plots if you fit the model on scaled data, too:
allhistory = historydf.merge(historydf_scaled,right_index=True, left_index=True, suffixes=["", "_scaled"])
allhistory[[c for c in allhistory.columns if "accuracy" in c]].plot();
allhistory[[c for c in allhistory.columns if "loss" in c]].plot(ylim=(0,1));

## Same model with functional API

In [ ]:
## import Input, Model from keras

In [ ]:
def create_model():

    tf.keras.backend.clear_session()

    ## create an input layer (necessary when using the functional API)
    input_layer = ...

    ## create the first layer as before, but with functional API
    layer1 = ...

    ## create the output layer as before, but with functional API
    output_layer = ...

    ## create the model that takes input_layer as input and output_layer as output.
    model = ...

    ## compile the model as before
    ...

    return model

In [ ]:
## just run it again to check it works
model, historydf = create_and_fit_model(X_train, y_train, X_val, y_val, num_epochs, num_batchsize)

## Evaluate on test set

In [ ]:
## optional: evaluate model on test set using its evaluate method
(model_loss, model_accuracy) = ...


In [ ]:
## optional: evaluate model_scaled on test set using its evaluate method
(model_scaled_loss, model_scaled_accuracy) = model_scaled.evaluate(X_test_scaled, y_test)


In [ ]:
## get the predictions of the model for the test set
y_test_pred = ...

## optional: get the predictions of model_scaled for the test set
#y_test_pred_scaled = ...

In [ ]:
## just run the code to get classes instead of floats between 0 and 1:

y_test_pred_cl = y_test_pred.round().flatten().astype(int)
#y_test_pred_cl_scaled = y_test_pred_scaled.round().flatten().astype(int)

In [ ]:
from sklearn import metrics

In [ ]:
## Print out the classification report for the test set predictions.
print("Non-scaled:")
print(...)

## Optional: do the same for the predictions of model_scaled.
#print("\nScaled:")
#print(...)